In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

load_dotenv()

translation_template = """
Translate the following sentence into {language}, return ONLY the translation, nothing else.

Sentence: {sentence}
"""

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

output_parser = StrOutputParser()
llm = ChatOpenAI(temperature=0.0, model="gpt-4-turbo", openai_api_key=OPENAI_API_KEY) 
translation_prompt = ChatPromptTemplate.from_template(translation_template)

translation_chain = (
    {"language": RunnablePassthrough(), "sentence": RunnablePassthrough()} 
    | translation_prompt
    | llm
    | output_parser
)

def translate(sentence, language="French"):
    data_input = {"language": language, "sentence": sentence}
    translation = translation_chain.invoke(data_input)
    return translation


# ElevenLabs For Voice Cloning & Voice Synthesis

In [16]:
from elevenlabs.client import ElevenLabs
from elevenlabs import play, stream

client = ElevenLabs()

def gen_dub(text):
    print("Generating audio...")
    audio = client.generate(
        text=text,
        voice="George", # Insert voice model here!
        model="eleven_multilingual_v2"
    )
    play(audio)

/Users/tententgc/Documents/GitHub/realtime-translator/translator/lib/python3.11/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)
/Users/tententgc/Documents/GitHub/realtime-translator/translator/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in SpeechHistoryItemResponse has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/Users/tententgc/Documents/GitHub/realtime-translator/translator/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_id" in Model has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.w

In [25]:
import assemblyai as aai


def on_open(session_opened: aai.RealtimeSessionOpened):
  "This function is called when the connection has been established."
  print("Session ID:", session_opened.session_id)

def on_data(transcript: aai.RealtimeTranscript):
  "This function is called when a new transcript has been received."
  if not transcript.text:
    return

  if isinstance(transcript, aai.RealtimeFinalTranscript):
    print(transcript.text, end="\r\n")
    print("Translating...")
    translation = translate(str(transcript.text))
    print(f"Translation: {translation}")
    gen_dub(translation)
  else:
    print(transcript.text, end="\r")
      
def on_error(error: aai.RealtimeError):
  "This function is called when the connection has been closed."
  print("An error occured:", error)

def on_close():
  "This function is called when the connection has been closed."
  print("Closing Session")
  
  
aai.settings.api_key=os.getenv("ASSEMBLYAI_API_KEY")

transcriber = aai.RealtimeTranscriber(
  on_data=on_data,
  on_error=on_error,
  sample_rate=44_100,
  on_open=on_open, # optional
  on_close=on_close, # optional
 
)

In [28]:
# Start the connection, likely have to restart kernal (runs better as full code in something like VSCode)
transcriber.connect()
microphone_stream = aai.extras.MicrophoneStream()
transcriber.stream(microphone_stream)

RuntimeError: threads can only be started once

In [ ]:
transcriber.close()